<a href="https://colab.research.google.com/github/burtonye/dahmakan_case/blob/master/Dahmakan_Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
url = 'https://raw.githubusercontent.com/burtonye/dahmakan_case/master/Copy%20of%20dahmakan%20analytics%20case%20study%20%5BBurton%20Ye%5D%20-%20Data.csv?token=ADMXF3J47NSNESC45QHYE426LKF3C'
df1 = pd.read_csv(url)

In [0]:
timedel = pd.to_datetime(df1['actual_last_delivered_time'])-pd.to_datetime(df1['actual_start_time'])
df1['time_elapsed']=timedel.apply(lambda x: x.seconds/3600)

# I: On Time Delivery%

Steps are: 
1. Group delivery metrics by rider_id 
2. Filter >20 rides as FT
3. Calculate On Time Delivery %



In [0]:
df = df1.groupby(['rider_id']).sum()

In [0]:
df_ft_riders = df[df['total_deliveries']>20]

data= df_ft_riders['ontime_deliveries']/df_ft_riders['completed_deliveries']
df_ft_riders['On_time_delivery%'] = data


# II: Calculate Deliveries per Hour

1. DPH = The no. of completed deliveries / total no. of hours worked"
2. Segment Good_Bad based on OTD%
3. Find average based on two groups

In [129]:
df_ft_riders['Good_Bad'] = ['Good' if x >= 0.9 else 'Bad' for x in df_ft_riders['On_time_delivery%']]

print("There are",df_ft_riders['Good_Bad'][df_ft_riders['Good_Bad'] == 'Good'].count(),"good riders")
print("There are",df_ft_riders['Good_Bad'][df_ft_riders['Good_Bad'] == 'Bad'].count(),"bad riders") 

There are 12 good riders
There are 157 bad riders


In [0]:
df_ft_riders['DPH'] = df_ft_riders['completed_deliveries']/df_ft_riders['time_elapsed']

In [131]:
print("The avg DPH is",round(df_ft_riders['DPH'][df_ft_riders['Good_Bad'] == 'Good'].mean(),2),"for good riders")
print("The avg DPH is",round(df_ft_riders['DPH'][df_ft_riders['Good_Bad'] == 'Bad'].mean(),2) ,"for bad  riders") 

The avg DPH is 5.32 for good riders
The avg DPH is 4.05 for bad  riders


# 4. Top 3 metrics to depict Good from Bad
1. Find mean of each of the provided columns in Full time riders dataframe
2. Identify metrics with large difference between Good and Bad


In [116]:
print(df_ft_riders[df_ft_riders['Good_Bad']=='Good'].mean())
print()
print(df_ft_riders[df_ft_riders['Good_Bad']=='Bad'].mean())
#cancelled_deliveries: 2.3 vs 4.1
#delayed_deliveries 5.8 vs 24.2
#delay_min per drop 68 vs 215
#avg_service_mins 73 vs 83


total_deliveries             103.166667
completed_deliveries         100.833333
cancelled_deliveries           2.333333
ontime_deliveries             95.000000
delayed_deliveries             5.833333
uncontactable_customers        2.083333
total_customers              101.916667
total_meal                   174.166667
total_meal_revenue         36804.933333
delay_min_per_drop            68.083333
avg_rider_rating              80.088333
avg_service_mins              73.015000
time_elapsed                  18.070833
On_time_delivery%              0.944642
DPH                            5.323406
dtype: float64

total_deliveries              91.917197
completed_deliveries          87.821656
cancelled_deliveries           4.095541
ontime_deliveries             63.579618
delayed_deliveries            24.242038
uncontactable_customers        2.878981
total_customers               90.713376
total_meal                   159.401274
total_meal_revenue         37938.310191
delay_min_per_drop      

# 6. Calculating Payment structure for riders
1. Define a function pricing to factor in the pricing table
2. Append that to dataframe
3. Calculate # of routes, # of meals, and # of hours
4. Divide by each of the factors and find the median for Ft and PT and then the overall average


In [0]:
ft_riders = list(df_ft_riders.index) #ft riders
def pricing(data,index=0):
  price = 15
  x = data.iloc[index]['ontime_deliveries']
  y = data.iloc[index]['completed_deliveries']
  price += 1 * y
  if x/y >= 0.95:
    price += 0.5*y
  elif x/y >= 0.9:
    price += 0.25*y
  elif x/y >= 0.8:
    price += 0
  else:
    price += -0.3*y
  if data.iloc[index]["rider_id"] in ft_riders:
    if y > 14:
      price +=75
    else:
      price +=40
  else:
    price +=0
  return price

pricing_list = []
for i in np.arange(df1.shape[0]):
  pricing_list.append(pricing(df1,i))
df1['pricing']= pd.Series(pricing_list)

In [118]:
df1['route'] = 1
df = df1.groupby(['rider_id']).sum()
df_pt_riders = df[df['total_deliveries']<20]
df_ft_riders = df[df['total_deliveries']>20]
df_ft_riders.head()

,total_deliveries,completed_deliveries,cancelled_deliveries,ontime_deliveries,delayed_deliveries,uncontactable_customers,total_customers,total_meal,total_meal_revenue,delay_min_per_drop,avg_rider_rating,avg_service_mins,time_elapsed,pricing,route
rider_id,,,,,,,,,,,,,,,
Rider 1,130,126,4,101,25,3,125,232,56045.8,291.0,104.12,113.00,26.866667,1463.5,24
Rider 10,264,259,5,232,27,2,258,443,102063.6,166.0,146.97,148.61,33.600000,2210.0,32
Rider 100,128,120,8,95,25,1,126,215,32853.5,285.0,103.36,99.07,27.483333,1341.1,22
Rider 101,62,57,5,40,17,3,62,92,15681.3,147.0,45.66,47.22,14.516667,665.9,11
Rider 102,79,69,10,57,12,3,76,123,20245.4,140.0,51.75,46.73,15.966667,735.1,12


In [0]:
#Pay/route 6a-6c

df_ft_riders['pay/route'] = df_ft_riders['pricing']/df_ft_riders['route']
six_a = np.median(df_ft_riders['pay/route'].tolist())

df_pt_riders['pay/route'] = df_pt_riders['pricing']/df_pt_riders['route']
six_b = np.median(df_pt_riders['pay/route'].tolist())

six_c = np.mean(df_pt_riders['pay/route'].tolist() + df_ft_riders['pay/route'].tolist())

# Pay/meal 6d-6f

df_ft_riders['pay/meal'] = df_ft_riders['pricing']/df_ft_riders['total_meal']
six_d = np.median(df_ft_riders['pay/meal'].tolist())

df_pt_riders['pay/meal'] = df_pt_riders['pricing']/df_pt_riders['total_meal']
six_e = np.median(df_pt_riders['pay/meal'].tolist())

six_f = np.mean(df_pt_riders['pay/meal'].tolist() + df_ft_riders['pay/meal'].tolist())

# Pay/hour 6g-6i

df_ft_riders['pay/hr'] = df_ft_riders['pricing']/df_ft_riders['time_elapsed']
six_g = np.median(df_ft_riders['pay/hr'].tolist())

df_pt_riders['pay/hr'] = df_pt_riders['pricing']/df_pt_riders['time_elapsed']
df_pt_riders = df_pt_riders.replace([np.inf], 0)  #edge case of Rider 20
six_h = np.median(df_pt_riders['pay/hr'].tolist())

six_i = np.mean(df_pt_riders['pay/hr'].tolist() + df_ft_riders['pay/hr'].tolist())

In [122]:
#Outputs for Question 6

print("Median FT pay/meal: ", round(six_a,2))
print("Median PT pay/meal: ", round(six_b,2))
print("Avg pay/meal: ", round(six_c,2))

print("Median FT pay/meal: ", round(six_d,2))
print("Median PT pay/meal: ", round(six_e,2))
print("Avg pay/meal: ", round(six_f,2))

print("median FT pay/hour: ", round(six_g,2))
print("Median PT pay/hour: ", round(six_h,2))
print("Avg pay/hour: ", round(six_i,2))

Median FT pay/meal:  60.38
Median PT pay/meal:  19.2
Avg pay/meal:  51.42
Median FT pay/meal:  6.38
Median PT pay/meal:  2.29
Avg pay/meal:  5.72
median FT pay/hour:  46.04
Median PT pay/hour:  13.28
Avg pay/hour:  39.67
